Импортируем нужные библиотеки.

In [1]:
import pandas as pd
import xgboost as xgb

Распакуйте архив с данными в папку,где находится этот jupyter notebook (baseline.ipynb). У вас будет папка data,  содержащая необходимые файлы. 

В данном соревновании перед вами ставится задача предсказания категории возраста, к которой принадлежит клиент банка, на основании его транзакций.
В обучающем наборе содержатся информация по транзакциям 30000 клиентов, она находится в файле **transactions_train.csv**. Правильная категория возраста для обучающего набора находится в файле **train_target.csv**.

Считаем данные по транзакциям и правильные ответы.

In [2]:
transactions_train=pd.read_csv('data/transactions_train.csv')

In [3]:
train_target=pd.read_csv('data/train_target.csv')

Посмотрим на данные.

In [4]:
transactions_train.head()

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341


* client_id - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* small_group - категория покупки
* amount_rur - сумма транзакции

In [5]:
train_target.head(5)

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3


* client_id - уникальный идентификатор клиента, соответствует полю client_id из транзакций
* bins - целевая переменная, которую нужно предсказать, это категория возраста клиента

Далее представлен простой вариант решения задачи. Вы можете решать соревнование используя совершенно другой подход.

Посчитаем по каждому клиенту самые простые аггрегационные признаки.

In [6]:
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [7]:
agg_features.head()

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


Посчитаем для каждого клиента количество транзакций по каждой категории.

In [8]:
counter_df_train=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()

In [9]:
cat_counts_train=counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')

In [10]:
cat_counts_train=cat_counts_train.fillna(0)

In [11]:
cat_counts_train.columns=['small_group_'+str(i) for i in cat_counts_train.columns]

In [12]:
cat_counts_train.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,...,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
client_id,,,,,,,,,,,,,,,,,,,,,
4,0.0,447.0,1.0,44.0,93.0,0.0,0.0,0.0,1.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,397.0,0.0,172.0,10.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,79.0,5.0,27.0,19.0,1.0,0.0,2.0,1.0,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,12.0,309.0,1.0,71.0,65.0,0.0,0.0,0.0,3.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2.0,423.0,0.0,59.0,23.0,3.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Далее соединим все файлы в один датафрейм с таргетом.

In [13]:
train=pd.merge(train_target,agg_features,on='client_id')

In [14]:
train=pd.merge(train,cat_counts_train.reset_index(),on='client_id')

In [15]:
train.head()

,client_id,bins,sum,mean,std,min,max,small_group_0,small_group_1,small_group_2,...,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь подгрузим тестовые данные для того, чтобы сделать предсказание. Проделаем с ними те же самые манипуляции, как и с обучающими данными.

In [16]:
transactions_test=pd.read_csv('data/transactions_test.csv')

test_id=pd.read_csv('data/test.csv')

In [17]:
agg_features_test=transactions_test.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [18]:
agg_features_test.head()

,client_id,sum,mean,std,min,max
0,0,17036.127,19.163247,40.561700,0.065,595.339
1,1,34748.964,47.666617,89.489278,0.298,1181.221
2,2,51188.069,68.433247,152.093601,0.043,2837.682
3,3,47975.203,67.857430,318.651653,0.043,6135.652
4,5,20059.100,21.803370,34.258433,0.043,439.902


In [19]:
counter_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].count()

In [20]:
cat_counts_test=counter_df_test.reset_index().pivot(index='client_id', columns='small_group',values='amount_rur')

In [21]:
cat_counts_test=cat_counts_test.fillna(0)

In [22]:
cat_counts_test.columns=['small_group_'+str(i) for i in cat_counts_test.columns]

In [23]:
cat_counts_test.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,...,small_group_192,small_group_193,small_group_194,small_group_195,small_group_196,small_group_197,small_group_198,small_group_200,small_group_201,small_group_202
client_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,226.0,1.0,36.0,9.0,0.0,0.0,0.0,2.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.0,326.0,0.0,40.0,56.0,0.0,0.0,0.0,0.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21.0,242.0,1.0,50.0,48.0,4.0,0.0,6.0,1.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,156.0,83.0,48.0,31.0,2.0,0.0,1.0,2.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,16.0,398.0,1.0,23.0,25.0,0.0,0.0,0.0,5.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
test=pd.merge(test_id,agg_features_test,on='client_id')

In [25]:
test=pd.merge(test,cat_counts_test.reset_index(),on='client_id')

В тесте не было некоторых категорий трат, поэтому для того, чтобы обучить модель, нам нужно объединить пространство признаков и train и test.

In [26]:
common_features=list(set(train.columns).intersection(set(test.columns)))

In [27]:
y_train=train['bins']
X_train=train[common_features]
X_test=test[common_features]

In [28]:
import catboost

In [29]:
clf = catboost.CatBoostClassifier(depth=8, iterations=5500)
clf.fit(X_train, y_train)

Learning rate set to 0.009605
0:	learn: 0.6897871	total: 478ms	remaining: 43m 50s
1:	learn: 0.6864275	total: 870ms	remaining: 39m 50s
2:	learn: 0.6830631	total: 1.3s	remaining: 39m 41s
3:	learn: 0.6800144	total: 1.66s	remaining: 37m 56s
4:	learn: 0.6771020	total: 2s	remaining: 36m 43s
5:	learn: 0.6743771	total: 2.36s	remaining: 35m 59s
6:	learn: 0.6713725	total: 2.69s	remaining: 35m 13s
7:	learn: 0.6685800	total: 3.01s	remaining: 34m 28s
8:	learn: 0.6659731	total: 3.32s	remaining: 33m 46s
9:	learn: 0.6634041	total: 3.62s	remaining: 33m 8s
10:	learn: 0.6608327	total: 3.93s	remaining: 32m 39s
11:	learn: 0.6583502	total: 4.22s	remaining: 32m 11s
12:	learn: 0.6558233	total: 4.55s	remaining: 31m 59s
13:	learn: 0.6533128	total: 4.86s	remaining: 31m 45s
14:	learn: 0.6508378	total: 5.18s	remaining: 31m 33s
15:	learn: 0.6486398	total: 5.49s	remaining: 31m 21s
16:	learn: 0.6463423	total: 5.8s	remaining: 31m 9s
17:	learn: 0.6439128	total: 6.08s	remaining: 30m 52s
18:	learn: 0.6417389	total: 6.4s	

155:	learn: 0.5233986	total: 45.4s	remaining: 25m 55s
156:	learn: 0.5229949	total: 45.8s	remaining: 25m 58s
157:	learn: 0.5226390	total: 46.1s	remaining: 25m 58s
158:	learn: 0.5222239	total: 46.4s	remaining: 25m 57s
159:	learn: 0.5219909	total: 46.6s	remaining: 25m 56s
160:	learn: 0.5216418	total: 46.9s	remaining: 25m 55s
161:	learn: 0.5213298	total: 47.1s	remaining: 25m 53s
162:	learn: 0.5211289	total: 47.4s	remaining: 25m 52s
163:	learn: 0.5208322	total: 47.7s	remaining: 25m 53s
164:	learn: 0.5204645	total: 48s	remaining: 25m 53s
165:	learn: 0.5201918	total: 48.4s	remaining: 25m 54s
166:	learn: 0.5198608	total: 48.7s	remaining: 25m 55s
167:	learn: 0.5196768	total: 49.1s	remaining: 25m 56s
168:	learn: 0.5194666	total: 49.4s	remaining: 25m 58s
169:	learn: 0.5192565	total: 49.7s	remaining: 25m 57s
170:	learn: 0.5190143	total: 50s	remaining: 25m 57s
171:	learn: 0.5187238	total: 50.3s	remaining: 25m 58s
172:	learn: 0.5185250	total: 50.6s	remaining: 25m 59s
173:	learn: 0.5182442	total: 51s

KeyboardInterrupt: 

Обучим xgboost на текущих признаках.

In [30]:
param={'objective':'multi:softprob','num_class':4,'n_jobs':4,'seed':42}

In [31]:
%%time
model=xgb.XGBClassifier(**param,n_estimators=300)
model.fit(X_train,y_train)

CPU times: user 4min 10s, sys: 1.21 s, total: 4min 11s
Wall time: 4min 17s


Сделаем предсказание.

In [32]:
pred=model.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [33]:
pred

array([0, 2, 3, ..., 2, 2, 3])

На public лидерборде такое предсказание должно дать качество 0.6118.

### Подготовим файл для отправки в систему

In [34]:
submission = pd.DataFrame({'bins': pred}, index=test.client_id)
submission.head()

,bins
client_id,
28571,0
27046,2
13240,3
19974,0
10505,1


Сохраняем прогноз на диск в папку submissions. Имя прогноза соответсвует дате и времени его создания, закодированными с помощью timestamp.

In [35]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv(submission_path, index=True)

submissions/1574086871.csv


Теперь все готово! Можно отправлять решение.